In [1]:
from enum import Enum, unique
import toml
import numpy as np
from rich.tree import Tree
import pandas as pd

import sys
sys.path.append('../src')
from sysloss.elements import *
from sysloss.elements import ElementTypes, _get_opt, _get_mand, RS_DEFAULT, LIMITS_DEFAULT

In [103]:
import rustworkx as rx
import numpy as np
from rich.tree import Tree
import json
import pandas as pd

def system_from_file(fname: str):
    """Generate system from json"""
    with open(fname, "r") as f:
        sys = json.load(f)
    # add root
    name = _get_mand(sys['root']['params'], 'name')
    vo = _get_mand(sys['root']['params'], "vo")
    rs = _get_opt(sys['root']['params'], "rs", RS_DEFAULT)
    lim = _get_opt(sys['root'], "limits", LIMITS_DEFAULT)
    s =  System(sys['name'], Source(name, vo=vo, rs=rs, limits=lim))
    # add childs
    if sys['childs'] != {}:
        for p in sys['childs']:
            print(p)
            for c in sys['childs'][p]:
                cname = _get_mand(c['params'], 'name')
                print("  "+cname)
                limits = _get_opt(c, 'limits', LIMITS_DEFAULT)
                iq = _get_opt(c['params'], 'iq', 0.0)
                if c['type'] == "CONVERTER":
                    vo = _get_mand(c['params'], 'vo')
                    eff = _get_mand(c['params'], 'eff')
                    s.add_element(p, element=Converter(cname, vo=vo, eff=eff, iq=iq, limits=limits))
                elif c['type'] == "LINREG":
                    vo = _get_mand(c['params'], 'vo')
                    vdrop = _get_opt(c['params'], 'vdrop', 0.0)
                    s.add_element(p, element=LinReg(cname, vo=vo, vdrop=vdrop, iq=iq, limits=limits))
                elif c['type'] == "LOSS":
                    rs = _get_mand(c['params'], 'rs')
                    vdrop = _get_mand(c['params'], 'vdrop')
                    s.add_element(p, element=Loss(cname, rs=rs, vdrop=vdrop, limits=limits))
                elif c['type'] == "LOAD":
                    if 'pwr' in c['params']:
                        pwr = _get_mand(c['params'], 'pwr')
                        s.add_element(p, element=PLoad(cname, pwr=pwr, limits=limits))
                    else:
                        ii = _get_mand(c['params'], 'ii')
                        s.add_element(p, element=ILoad(cname, ii=ii, limits=limits))
        return s

class System:
    """System to be analyzed."""

    def __init__(self, name: str, source):
        self.g = None
        if not isinstance(source, Source):
            raise ValueError("Error: First element of system must be a source!")
        else:
            self.g = rx.PyDAG(check_cycle=True, multigraph=False, attrs={})
            pidx = self.g.add_node(source)
            self.g.attrs["name"] = name
            self.g.attrs["nodes"] = {}
            self.g.attrs["nodes"][source.params["name"]] = pidx

    @classmethod
    def from_file(cls, fname: str):
        """Load system from json file"""
        with open(fname, "r") as f:
            sys = json.load(f)

        # add root
        name = _get_mand(sys["root"]["params"], "name")
        vo = _get_mand(sys["root"]["params"], "vo")
        rs = _get_opt(sys["root"]["params"], "rs", RS_DEFAULT)
        lim = _get_opt(sys["root"], "limits", LIMITS_DEFAULT)
        self = cls(sys["name"], Source(name, vo=vo, rs=rs, limits=lim))
        # add childs
        if sys["childs"] != {}:
            for p in sys["childs"]:
                # print(p)
                for c in sys["childs"][p]:
                    cname = _get_mand(c["params"], "name")
                    # print("  " + cname)
                    limits = _get_opt(c, "limits", LIMITS_DEFAULT)
                    iq = _get_opt(c["params"], "iq", 0.0)
                    if c["type"] == "CONVERTER":
                        vo = _get_mand(c["params"], "vo")
                        eff = _get_mand(c["params"], "eff")
                        self.add_element(
                            p,
                            element=Converter(
                                cname, vo=vo, eff=eff, iq=iq, limits=limits
                            ),
                        )
                    elif c["type"] == "LINREG":
                        vo = _get_mand(c["params"], "vo")
                        vdrop = _get_opt(c["params"], "vdrop", 0.0)
                        self.add_element(
                            p,
                            element=LinReg(
                                cname, vo=vo, vdrop=vdrop, iq=iq, limits=limits
                            ),
                        )
                    elif c["type"] == "LOSS":
                        rs = _get_mand(c["params"], "rs")
                        vdrop = _get_mand(c["params"], "vdrop")
                        self.add_element(
                            p, element=Loss(cname, rs=rs, vdrop=vdrop, limits=limits)
                        )
                    elif c["type"] == "LOAD":
                        if "pwr" in c["params"]:
                            pwr = _get_mand(c["params"], "pwr")
                            self.add_element(
                                p, element=PLoad(cname, pwr=pwr, limits=limits)
                            )
                        elif "rs" in c["params"]:
                            rs = _get_mand(c["params"], "rs")
                            self.add_element(
                                p, element=RLoad(cname, rs=rs, limits=limits)
                            )
                        else:
                            ii = _get_mand(c["params"], "ii")
                            self.add_element(
                                p, element=ILoad(cname, ii=ii, limits=limits)
                            )

        return self

    def __get_index(self, name: str):
        """Get node index from element name"""
        if name in self.g.attrs["nodes"]:
            return self.g.attrs["nodes"][name]

        return -1

    def __chk_parent(self, parent: str):
        """Check if parent exists"""
        if not parent in self.g.attrs["nodes"].keys():
            raise ValueError('Error: Parent name "{}" not found!'.format(parent))

        return True

    def __chk_name(self, name: str):
        """Check if element name is valid"""
        # check if name exists
        if name in self.g.attrs["nodes"].keys():
            raise ValueError('Error: Element name "{}" is already used!'.format(name))

        return True

    def __get_childs_tree(self):
        """Get dict of parent/childs"""
        # if rev == True:
        #    childs = list(reversed(rx.bfs_successors(self.g, 0)))
        # else:
        childs = list(rx.bfs_successors(self.g, 0))
        cdict = {}
        for c in childs:
            cs = []
            for l in c[1]:
                cs += [self.g.attrs["nodes"][l.params["name"]]]
            cdict[self.g.attrs["nodes"][c[0].params["name"]]] = cs
        return cdict

    def __get_nodes(self):
        """Get list of nodes in system"""
        nodes = []
        for n in self.g.nodes():
            nodes += [self.g.attrs["nodes"][n.params["name"]]]
        return sorted(nodes)

    def __get_childs(self):
        """Get list of children of each node"""
        nodes = self.__get_nodes()
        cs = list(-np.ones(max(nodes) + 1, dtype=np.int32))
        for n in nodes:
            if self.g.out_degree(n) > 0:
                ind = [i for i in self.g.successor_indices(n)]
                cs[n] = ind
        return cs

    def __get_parents(self):
        """Get list of parent of each node"""
        nodes = self.__get_nodes()
        ps = list(-np.ones(max(nodes) + 1, dtype=np.int32))
        for n in nodes:
            if self.g.in_degree(n) > 0:
                ind = [i for i in self.g.predecessor_indices(n)]
                ps[n] = ind
        return ps

    def __get_topo_sort(self):
        """Get nodes topological sorted"""
        tps = rx.topological_sort(self.g)
        return [n for n in tps]

    def __sys_vars(self):
        """Get system variable lists"""
        vn = max(self.__get_nodes()) + 1  # highest node index + 1
        v = list(np.zeros(vn))  # voltages
        i = list(np.zeros(vn))  # currents
        return v, i

    def __make_rtree(self, adj, node):
        """Create Rich tree"""
        tree = Tree(node)
        for child in adj.get(node, []):
            tree.add(self.__make_rtree(adj, child))
        return tree

    def add_element(self, parent: str, *, element):
        """Add element to system"""
        # check that parent exists
        self.__chk_parent(parent)
        # check that element name is unique
        self.__chk_name(element.params["name"])

        pidx = self.__get_index(parent)

        # check that parent allows element type as child
        if not element.element_type in self.g[pidx].child_types:
            raise ValueError(
                "Error: Parent does not allow child of type {}!".format(
                    element.element_type.name
                )
            )
        cidx = self.g.add_child(pidx, element, None)
        self.g.attrs["nodes"][element.params["name"]] = cidx

    def change_element(self, name: str, *, element):
        """Replace element with a new one"""
        # if element name changes, check that it is unique
        if name != element.params["name"]:
            self.__chk_name(element.params["name"])

        eidx = self.__get_index(name)
        # check that parent allows element type as child
        parents = self.__get_parents()
        if parents[eidx] != -1:
            if not element.element_type in self.g[parents[eidx][0]].child_types:
                raise ValueError(
                    "Error: Parent does not allow child of type {}!".format(
                        element.element_type.name
                    )
                )
        self.g[eidx] = element
        # replace node name in graph dict
        del [self.g.attrs["nodes"][name]]
        self.g.attrs["nodes"][element.params["name"]] = eidx

    def del_element(self, name: str, *, del_childs: bool = True):
        eidx = self.__get_index(name)
        if eidx == -1:
            raise ValueError("Error: Element name does not exist!")
        parents = self.__get_parents()
        if parents[eidx] == -1:
            raise ValueError("Error: Cannot delete source node!")
        childs = self.__get_childs()
        # if not leaf, check if child type is allowed by parent type (not possible?)
        # if leaves[eidx] == 0:
        #     for c in childs[eidx]:
        #         if not self.g[c].element_type in self.g[parents[eidx]].child_types:
        #             raise ValueError(
        #                 "Error: Parent and child of element are not compatible!"
        #             )
        # delete childs first if selected
        if del_childs:
            for c in rx.descendants(self.g, eidx):
                print(c, eidx)
                del [self.g.attrs["nodes"][self.g[c].params["name"]]]
                self.g.remove_node(c)
        # delete node
        self.g.remove_node(eidx)
        del [self.g.attrs["nodes"][name]]
        # restore links between new parent and childs, unless deleted
        if not del_childs:
            if childs[eidx] != -1:
                for c in childs[eidx]:
                    self.g.add_edge(parents[eidx][0], c, None)

    def tree(self, name=""):
        """Print tree structure, starting from node name"""
        if not name == "":
            if not name in self.g.attrs.keys():
                raise ValueError("Error: Element name is not valid!")
        else:
            root = self.g[0].params["name"]

        adj = rx.bfs_successors(self.g, self.g.attrs["nodes"][root])
        ndict = {}
        for i in adj:
            c = []
            for j in i[1]:
                c += [j.params["name"]]
            ndict[i[0].params["name"]] = c
        return self.__make_rtree(ndict, root)

    def __sys_init(self):
        """Create vectors of init values for solver"""
        v, i = self.__sys_vars()
        for n in self.__get_nodes():
            v[n] = self.g[n]._get_outp_voltage()
            i[n] = self.g[n]._get_inp_current()
        return v, i

    def __fwd_prop(self, v: float, i: float):
        """Forward propagation of voltages"""
        vo, _ = self.__sys_vars()
        # update output voltages (per node)
        for n in self.__topo_nodes:
            p = self.__parents[n]
            if self.__childs[n] == -1:  # leaf
                if p == -1:  # root
                    vo[n] = self.g[n]._solv_outp_volt(0.0, 0.0, 0.0)
                else:
                    vo[n] = self.g[n]._solv_outp_volt(v[p[0]], i[n], 0.0)
            else:
                # add currents into childs
                isum = 0
                for c in self.__childs[n]:
                    isum += i[c]
                if n == 0:  # root
                    vo[n] = self.g[n]._solv_outp_volt(0.0, 0.0, isum)
                else:
                    vo[n] = self.g[n]._solv_outp_volt(v[p[0]], i[n], isum)
        return vo

    def __back_prop(self, v: float, i: float):
        """Backward propagation of currents"""
        _, ii = self.__sys_vars()
        # update input currents (per node)
        for n in self.__topo_nodes[::-1]:
            p = self.__parents[n]
            if self.__childs[n] == -1:  # leaf
                if p == -1:  # root
                    ii[n] = self.g[n]._solv_inp_curr(v[n], 0.0, 0.0)
                else:
                    ii[n] = self.g[n]._solv_inp_curr(v[p[0]], 0.0, 0.0)
            else:
                isum = 0.0
                for c in self.__childs[n]:
                    isum += i[c]
                if p == -1:  # root
                    ii[n] = self.g[n]._solv_inp_curr(v[n], v[n], isum)
                else:
                    ii[n] = self.g[n]._solv_inp_curr(v[p[0]], v[n], isum)

        return ii

    def __rel_update(self):
        """Update lists with element relationships"""
        self.__parents = self.__get_parents()
        self.__childs = self.__get_childs()
        self.__topo_nodes = self.__get_topo_sort()

    def __get_parent_name(self, node):
        """Get parent name of node"""
        if self.__parents[node] == -1:
            return ""

        return self.g[self.__parents[node][0]].params["name"]

    def solve(self, *, vtol=1e-5, itol=1e-6, maxiter=1000, quiet=True):
        """Analyze system"""
        self.__rel_update()
        # initial condition
        v, i = self.__sys_init()
        # solve system function
        iters = 1
        while iters <= maxiter:
            vi = self.__fwd_prop(v, i)
            ii = self.__back_prop(vi, i)
            if np.allclose(np.array(v), np.array(vi), rtol=vtol) and np.allclose(
                np.array(i), np.array(ii), rtol=itol
            ):
                if not quiet:
                    print("Tolerances met after {} iterations".format(iters))
                break
            v, i = vi, ii
            iters += 1

        if iters > maxiter:
            print("Analysis aborted after {} iterations".format(iters - 1))
            return None

        # calculate results for each node
        names, parent, typ, pwr, loss = [], [], [], [], []
        eff, warn, vsi, iso = [], [], [], []
        for n in self.__topo_nodes:  # [vi, vo, ii, io]
            names += [self.g[n].params["name"]]
            vi = v[n]
            vo = v[n]
            ii = i[n]
            io = i[n]
            p = self.__parents[n]

            if p == -1:  # root
                vi = v[n] + self.g[n].params["rs"] * ii
            elif self.__childs[n] == -1:  # leaf
                vi = v[p[0]]
                io = 0.0
            else:
                io = 0.0
                for c in self.__childs[n]:
                    io += i[c]
                vi = v[p[0]]
            parent += [self.__get_parent_name(n)]
            p, l, e = self.g[n]._solv_pwr_loss(vi, vo, ii, io)
            pwr += [p]
            loss += [l]
            eff += [e]
            typ += [self.g[n].element_type.name]
            warn += [self.g[n]._solv_get_warns(vi, vo, ii, io)]
            vsi += [vi]
            iso += [io]

        # remove unused node indices
        vso, isi = [], []
        for n in range(len(v)):
            if n in self.__topo_nodes:
                vso += [v[n]]
                isi += [i[n]]

        # report
        res = {}
        res["Element"] = names
        res["Type"] = typ
        res["Parent"] = parent
        res["Vin (V)"] = vsi
        res["Vout (V)"] = vso
        res["Iin (A)"] = isi
        res["Iout (A)"] = iso
        res["Power (W)"] = pwr  # [res['Power (W)'].sum() - res['Power (W)'][0]]
        res["Loss (W)"] = loss  # [res['Loss (W)'].sum()]
        res["Efficiency (%)"] = eff
        res["Warnings"] = warn
        df = pd.DataFrame(res)
        tpwr = abs(vsi[0] * i[0])
        tloss = df["Loss (W)"].sum()
        w = "None"
        if sum(df["Warnings"] != "") > 0:
            w = "Yes"
        eff = 0.0
        if tpwr > 0.0:
            eff = (tpwr - tloss) / tpwr
        df.loc[len(df)] = [
            "System total",
            "",
            "",
            vsi[0],
            0.0,
            i[0],
            0.0,
            tpwr,
            tloss,
            eff,
            w,
        ]

        return df

    def params(self, limits=False):
        """Return element parameters"""
        self.__rel_update()
        names, typ, parent, vo, vdrop = [], [], [], [], []
        iq, rs, eff, ii, pwr = [], [], [], [], []
        lii, lio, lvi, lvo = [], [], [], []
        for n in self.__topo_nodes:
            names += [self.g[n].params["name"]]
            typ += [self.g[n].element_type.name]
            _vo, _vdrop, _iq, _rs, _eff, _ii, _pwr = "", "", "", "", "", "", ""
            if self.g[n].element_type == ElementTypes.SOURCE:
                _vo = self.g[n].params["vo"]
                _rs = self.g[n].params["rs"]
            elif self.g[n].element_type == ElementTypes.LOAD:
                if "pwr" in self.g[n].params:
                    _pwr = self.g[n].params["pwr"]
                elif "rs" in self.g[n].params:
                    _rs = self.g[n].params["rs"]
                else:
                    _ii = self.g[n].params["ii"]
            elif self.g[n].element_type == ElementTypes.CONVERTER:
                _vo = self.g[n].params["vo"]
                _iq = self.g[n].params["iq"]
                _eff = self.g[n].params["eff"]
            elif self.g[n].element_type == ElementTypes.LINREG:
                _vo = self.g[n].params["vo"]
                _vdrop = self.g[n].params["vdrop"]
                _iq = self.g[n].params["iq"]
            elif self.g[n].element_type == ElementTypes.LOSS:
                _vdrop = self.g[n].params["vdrop"]
                _rs = self.g[n].params["rs"]
            vo += [_vo]
            vdrop += [_vdrop]
            iq += [_iq]
            rs += [_rs]
            eff += [_eff]
            ii += [_ii]
            pwr += [_pwr]
            parent += [self.__get_parent_name(n)]
            if limits:
                lii += [self.g[n].limits["ii"]]
                lio += [self.g[n].limits["io"]]
                lvi += [self.g[n].limits["vi"]]
                lvo += [self.g[n].limits["vo"]]
        # report
        res = {}
        res["Element"] = names
        res["Type"] = typ
        res["Parent"] = parent
        res["vo (V)"] = vo
        res["vdrop (V)"] = vdrop
        res["iq (A)"] = iq
        res["rs (Ohm)"] = rs
        res["eff (%)"] = eff
        res["ii (A)"] = ii
        res["pwr (W)"] = pwr
        if limits:
            res["vi limits (V)"] = lvi
            res["vo limits (V)"] = lvo
            res["ii limits (A)"] = lii
            res["io limits (A)"] = lio
        return pd.DataFrame(res)

    def save(self, fname, *, indent=4):
        """Save system as json file"""
        self.__rel_update()
        sys = {
            "name": self.g.attrs["name"],
            "root": {
                "type": self.g[0].element_type.name,
                "params": self.g[0].params,
                "limits": self.g[0].limits,
            },
        }
        tree = self.__get_childs_tree()
        cdict = {}
        if tree != {}:
            for e in tree:
                childs = []
                for c in tree[e]:
                    childs += [
                        {
                            "type": self.g[c].element_type.name,
                            "params": self.g[c].params,
                            "limits": self.g[c].limits,
                        }
                    ]
                cdict[self.g[e].params["name"]] = childs
        sys["childs"] = cdict

        with open(fname, "w") as f:
            json.dump(sys, f, indent=indent)


![case1](case_1.png)

In [104]:
case1 = System('Bluetooth sensor', Source('3V coin', vo=3, rs=13e-3))
case1.add_element('3V coin', element=Converter('1.8V buck', vo=1.8, eff=0.87, iq=12e-6))
#case1.add_element('1.8V buck', element=Loss('Resistor2', rs=5.0))
case1.add_element('1.8V buck', element=PLoad('MCU', pwr=27e-3))
case1.add_element('3V coin', element=Converter('5V boost', vo=5, eff=0.91, iq=42e-6))
case1.add_element('5V boost', element=ILoad('Sensor', ii=15e-3))
case1.add_element(parent='5V boost', element=Loss('RC filter', rs=33.0))
case1.add_element('RC filter', element=LinReg('LDO 2.5V', vo=2.5, vdrop=0.27, iq=150e-6))
case1.add_element('LDO 2.5V', element=PLoad('ADC', pwr=15e-3))
case1.add_element('5V boost', element=RLoad('Res divider', rs=200e3))
#case1.g.attrs
case1.tree()

3V coin
├── 5V boost
│   ├── Res divider
│   ├── RC filter
│   │   └── LDO 2.5V
│   │       └── ADC
│   └── Sensor
└── 1.8V buck
    └── MCU

In [105]:
df = case1.solve(quiet=False)
df

Tolerances met after 8 iterations


,Element,Type,Parent,Vin (V),Vout (V),Iin (A),Iout (A),Power (W),Loss (W),Efficiency (%),Warnings
0,3V coin,SOURCE,,3.000000,2.999361,0.049191,0.049191,0.147543,0.000031,0.999787,
1,5V boost,CONVERTER,3V coin,2.999361,1.800000,0.010359,0.021175,0.000000,0.010597,0.909016,
2,Res divider,LOAD,5V boost,5.000000,0.000000,0.015000,0.000000,0.000125,0.000000,100.000000,
3,RC filter,LOSS,5V boost,5.000000,5.000000,0.038832,0.006150,0.000000,0.001248,0.959410,
4,LDO 2.5V,LINREG,RC filter,4.797050,0.000000,0.015000,0.006000,0.000000,0.014502,0.508443,
5,ADC,LOAD,LDO 2.5V,2.500000,4.797050,0.006150,0.000000,0.015000,0.000000,100.000000,
6,Sensor,LOAD,5V boost,5.000000,2.500000,0.006150,0.000000,0.075000,0.000000,100.000000,
7,1.8V buck,CONVERTER,3V coin,2.999361,0.000000,0.006000,0.015000,0.000000,0.004070,0.868992,
8,MCU,LOAD,1.8V buck,1.800000,0.000000,0.000025,0.000000,0.027000,0.000000,100.000000,
9,System total,,,3.000000,0.000000,0.049191,0.000000,0.147574,0.030449,0.793669,None


In [102]:
%timeit case1.solve()

2.54 ms ± 13.1 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [106]:
#case1.save('test2.json')
case1.params(limits=True)

,Element,Type,Parent,vo (V),vdrop (V),iq (A),rs (Ohm),eff (%),ii (A),pwr (W),vi limits (V),vo limits (V),ii limits (A),io limits (A)
0,3V coin,SOURCE,,3,,,0.013,,,,"[0.0, 1000000.0]","[0.0, 1000000.0]","[0.0, 1000000.0]","[0.0, 1000000.0]"
1,5V boost,CONVERTER,3V coin,5,,0.000042,,0.91,,,"[0.0, 1000000.0]","[0.0, 1000000.0]","[0.0, 1000000.0]","[0.0, 1000000.0]"
2,Res divider,LOAD,5V boost,,,,200000.0,,,,"[0.0, 1000000.0]","[0.0, 1000000.0]","[0.0, 1000000.0]","[0.0, 1000000.0]"
3,RC filter,LOSS,5V boost,,0.0,,33.0,,,,"[0.0, 1000000.0]","[0.0, 1000000.0]","[0.0, 1000000.0]","[0.0, 1000000.0]"
4,LDO 2.5V,LINREG,RC filter,2.5,0.27,0.00015,,,,,"[0.0, 1000000.0]","[0.0, 1000000.0]","[0.0, 1000000.0]","[0.0, 1000000.0]"
5,ADC,LOAD,LDO 2.5V,,,,,,,0.015,"[0.0, 1000000.0]","[0.0, 1000000.0]","[0.0, 1000000.0]","[0.0, 1000000.0]"
6,Sensor,LOAD,5V boost,,,,,,0.015,,"[0.0, 1000000.0]","[0.0, 1000000.0]","[0.0, 1000000.0]","[0.0, 1000000.0]"
7,1.8V buck,CONVERTER,3V coin,1.8,,0.000012,,0.87,,,"[0.0, 1000000.0]","[0.0, 1000000.0]","[0.0, 1000000.0]","[0.0, 1000000.0]"
8,MCU,LOAD,1.8V buck,,,,,,,0.027,"[0.0, 1000000.0]","[0.0, 1000000.0]","[0.0, 1000000.0]","[0.0, 1000000.0]"


In [15]:
#case1.change_element(name='MCU', element=Load('Big MCU', pwr=130e-3))
#case1.change_element(name='MCU', element=Loss('Resistor1', res=133.0))
#case1.change_element(name='1.8V buck', element=Loss('Resistor2', res=7.8))
#case1.add_element('ADC', element='dummy')
case1.del_element(name='5V boost', del_childs=False)
case1.tree()

3V coin
├── Sensor
├── RC filter
│   └── LDO 2.5V
│       └── ADC
├── Res divider
└── 1.8V buck
    └── MCU

In [16]:
case1.solve()

,Element,Type,Parent,Vin (V),Vout (V),Iin (A),Iout (A),Power (W),Loss (W),Efficiency (%),Warnings
0,3V coin,SOURCE,,3.00000,2.99959,0.031523,0.031523,0.094557,0.000013,0.999863,
1,1.8V buck,CONVERTER,3V coin,2.99959,1.80000,0.010358,0.015000,0.000000,0.004070,0.868992,
2,MCU,LOAD,1.8V buck,1.80000,0.00000,0.015000,0.000000,0.027000,0.000000,100.000000,
3,Sensor,LOAD,3V coin,2.99959,0.00000,0.015000,0.000000,0.044994,0.000000,100.000000,
4,RC filter,LOSS,3V coin,2.99959,2.79664,0.006150,0.006150,0.000000,0.001248,0.932341,
5,LDO 2.5V,LINREG,RC filter,2.79664,2.50000,0.006150,0.006000,0.000000,0.002199,0.872127,
6,ADC,LOAD,LDO 2.5V,2.50000,0.00000,0.006000,0.000000,0.015000,0.000000,100.000000,
7,Res divider,LOAD,3V coin,2.99959,0.00000,0.000015,0.000000,0.000045,0.000000,100.000000,
8,System total,,,3.00000,0.00000,0.031523,0.000000,0.094570,0.007531,0.920367,None


| Element | vo (V)   | vdrop (V) | iq (A) | rs (Ohm) | eff (%) | ii (A) | pwr (W) |
|:--------:|:--------:|:--------:|:--------:|:--------:|:--------:|:--------:|:--------:|
|  **Source**  | # |     |   | (#) |  |  |  |
|  **ILoad**   |     |     || || #||
|  **PLoad**   |     |    ||||| #|
|  **RLoad**   |     |    ||#||| |
|  **Converter**   |  #   |     | (#)||#|||
|  **LinReg**  |  # | (#)   | (#) |||||
|  **Loss**  |     | #   || # ||||

In [128]:
case2 = System('Case2', Source('3V coin', vo=3, rs=13e-3))
#case2.add_element('3V coin', element=Loss('RC filter', rs=33.0))
#case2.solve(quiet=False)
case2.save('test3.json')
case2.solve()

,Element,Type,Parent,Vin (V),Vout (V),Iin (A),Iout (A),Power (W),Loss (W),Efficiency (%),Warnings
0,3V coin,SOURCE,,3.0,3.0,0.0,0.0,0.0,0.0,100.0,
1,System total,,,3.0,0.0,0.0,0.0,0.0,0.0,0.0,None


In [60]:
sys = {'name': 'system name', 'root':
       {'type': 'SOURCE', 'params': {'name': '3V coin', 'vo': 3, 'rs': 0.013}, 
        'limits':{'vi': [0.0, 1000000.0], 'vo': [0.0, 1000000.0], 'ii': [0.0, 1000000.0], 'io': [0.0, 1000000.0]}}}
sys

{'name': 'system name',
 'root': {'type': 'SOURCE',
  'params': {'name': '3V coin', 'vo': 3, 'rs': 0.013},
  'limits': {'vi': [0.0, 1000000.0],
   'vo': [0.0, 1000000.0],
   'ii': [0.0, 1000000.0],
   'io': [0.0, 1000000.0]}}}

In [30]:
root = case1.g[0].params['name']
adj = rx.bfs_successors(case1.g, case1.g.attrs[root])
ndict = {}
for i in adj:
    c = []
    for j in i[1]:
        c += [j.params['name']]
    ndict[i[0].params['name']] = c
ndict

{'3V coin': ['5V boost', '1.8V buck'],
 '5V boost': ['RC filter', 'Sensor'],
 '1.8V buck': ['MCU'],
 'RC filter': ['LDO 2.5V'],
 'LDO 2.5V': ['ADC']}

In [236]:
c3 = System.from_file('test3.json')
c3.solve(quiet=False)

Tolerances met after 1 iterations


,Element,Type,Parent,Vin (V),Vout (V),Iin (A),Iout (A),Power (W),Loss (W),Efficiency (%),Warnings
0,3V coin,SOURCE,,3.0,3.0,0.0,0.0,0.0,0.0,100.0,
1,System total,,,3.0,0.0,0.0,0.0,0.0,0.0,0.0,None


In [188]:
c3.params()

,Element,Type,Parent,vo (V),vdrop (V),iq (A),rs (Ohm),eff (%),ii (A),pwr (W)
0,3V coin,SOURCE,,3,,,0.013,,,


In [58]:
tree = {0: [3, 1], 3: [5, 4], 1: [2], 5: [6], 6: [7]}
cdict = {}
for e in tree:
    print(case1.g[e].params['name'], tree[e])
    childs = []
    for c in tree[e]:
        childs += [{'type': case1.g[c].element_type.name, 'params':case1.g[c].params, 'limits':case1.g[c].limits}]
    cdict[case1.g[e].params['name']] = childs

3V coin [3, 1]
5V boost [5, 4]
1.8V buck [2]
RC filter [6]
LDO 2.5V [7]


In [261]:
case2 = System('Bluetooth sensor', Source('3V coin', vo=3, rs=13e-3))
#case2.save('test3.json')
case2.solve()

,Element,Type,Parent,Vin (V),Vout (V),Iin (A),Iout (A),Power (W),Loss (W),Efficiency (%),Warnings
0,3V coin,SOURCE,,3.0,3.0,0.0,0.0,0.0,0.0,100.0,
1,System total,,,3.0,0.0,0.0,0.0,0.0,0.0,0.0,None


In [66]:
case1.g.attrs

{'3V coin': 0,
 '1.8V buck': 1,
 'MCU': 2,
 '5V boost': 3,
 'Sensor': 4,
 'RC filter': 5,
 'LDO 2.5V': 6,
 'ADC': 7}

In [10]:
case1b = System.from_file('test2.json')
#case3 = system_from_file('test2.json')
case1b.g.attrs

3V coin
  5V boost
  1.8V buck
5V boost
  RC filter
  Sensor
1.8V buck
  MCU
RC filter
  LDO 2.5V
LDO 2.5V
  ADC


{'name': 'Bluetooth sensor',
 'nodes': {'3V coin': 0,
  '5V boost': 1,
  '1.8V buck': 2,
  'RC filter': 3,
  'Sensor': 4,
  'MCU': 5,
  'LDO 2.5V': 6,
  'ADC': 7}}

In [11]:
case1b.solve(quiet=False)

Tolerances met after 8 iterations


,Element,Type,Parent,Vin (V),Vout (V),Iin (A),Iout (A),Power (W),Loss (W),Efficiency (%),Warnings
0,3V coin,SOURCE,,3.000000,2.999361,0.049146,0.049146,0.147405,0.000031,0.999787,
1,5V boost,CONVERTER,3V coin,2.999361,5.000000,0.038787,0.021150,0.000000,0.010585,0.909015,
2,1.8V buck,CONVERTER,3V coin,2.999361,1.800000,0.010359,0.015000,0.000000,0.004070,0.868992,
3,RC filter,LOSS,5V boost,5.000000,4.797050,0.006150,0.006150,0.000000,0.001248,0.959410,
4,Sensor,LOAD,5V boost,5.000000,0.000000,0.015000,0.000000,0.075000,0.000000,100.000000,
5,MCU,LOAD,1.8V buck,1.800000,0.000000,0.015000,0.000000,0.027000,0.000000,100.000000,
6,LDO 2.5V,LINREG,RC filter,4.797050,2.500000,0.006150,0.006000,0.000000,0.014502,0.508443,
7,ADC,LOAD,LDO 2.5V,2.500000,0.000000,0.006000,0.000000,0.015000,0.000000,100.000000,
8,System total,,,3.000000,0.000000,0.049146,0.000000,0.147437,0.030437,0.793561,None


In [240]:
case3.params()

,Element,Type,Parent,vo (V),vdrop (V),iq (A),rs (Ohm),eff (%),ii (A),pwr (W)
0,3V coin,SOURCE,,3,,,0.013,,,
1,5V boost,CONVERTER,3V coin,5,,0.000042,,0.91,,
2,1.8V buck,CONVERTER,3V coin,1.8,,0.000012,,0.87,,
3,RC filter,LOSS,5V boost,,0.0,,33.0,,,
4,Sensor,LOAD,5V boost,,,,,,0.015,
5,MCU,LOAD,1.8V buck,,,,,,,0.027
6,LDO 2.5V,LINREG,RC filter,2.5,0.27,0.00015,,,,
7,ADC,LOAD,LDO 2.5V,,,,,,,0.015


In [242]:
case1.solve()

,Element,Type,Parent,Vin (V),Vout (V),Iin (A),Iout (A),Power (W),Loss (W),Efficiency (%),Warnings
0,3V coin,SOURCE,,3.000000,2.999718,0.021665,0.021665,0.064988,0.000006,0.999906,
1,1.8V buck,CONVERTER,3V coin,2.999718,1.800000,0.010358,0.015000,0.000000,0.004070,0.868992,
2,MCU,LOAD,1.8V buck,1.800000,0.000000,0.015000,0.000000,0.027000,0.000000,100.000000,
3,5V boost,CONVERTER,3V coin,2.999718,5.000000,0.011307,0.021150,0.000000,0.003167,0.906620,
4,Sensor,LOAD,5V boost,5.000000,0.000000,0.015000,0.000000,0.075000,0.000000,100.000000,
5,RC filter,LOSS,5V boost,5.000000,4.797050,0.006150,0.006150,0.000000,0.001248,0.959410,
6,LDO 2.5V,LINREG,RC filter,4.797050,2.500000,0.006150,0.006000,0.000000,0.014502,0.508443,
7,ADC,LOAD,LDO 2.5V,2.500000,0.000000,0.006000,0.000000,0.015000,0.000000,100.000000,
8,System total,,,3.000000,0.000000,0.021665,0.000000,0.064994,0.022994,0.646216,None


In [243]:
case1.params()

,Element,Type,Parent,vo (V),vdrop (V),iq (A),rs (Ohm),eff (%),ii (A),pwr (W)
0,3V coin,SOURCE,,3,,,0.013,,,
1,1.8V buck,CONVERTER,3V coin,1.8,,0.000012,,0.87,,
2,MCU,LOAD,1.8V buck,,,,,,,0.027
3,5V boost,CONVERTER,3V coin,5,,0.000042,,0.91,,
4,Sensor,LOAD,5V boost,,,,,,0.015,
5,RC filter,LOSS,5V boost,,0.0,,33.0,,,
6,LDO 2.5V,LINREG,RC filter,2.5,0.27,0.00015,,,,
7,ADC,LOAD,LDO 2.5V,,,,,,,0.015


In [244]:
case3.save('test2b.json')

In [245]:
case4 = System.from_file('test2b.json')
case4.solve()

3V coin
  1.8V buck
  5V boost
1.8V buck
  MCU
5V boost
  Sensor
  RC filter
RC filter
  LDO 2.5V
LDO 2.5V
  ADC


,Element,Type,Parent,Vin (V),Vout (V),Iin (A),Iout (A),Power (W),Loss (W),Efficiency (%),Warnings
0,3V coin,SOURCE,,3.000000,2.999718,0.021665,0.021665,0.064988,0.000006,0.999906,
1,1.8V buck,CONVERTER,3V coin,2.999718,1.800000,0.010358,0.015000,0.000000,0.004070,0.868992,
2,5V boost,CONVERTER,3V coin,2.999718,5.000000,0.011307,0.021150,0.000000,0.003167,0.906620,
3,MCU,LOAD,1.8V buck,1.800000,0.000000,0.015000,0.000000,0.027000,0.000000,100.000000,
4,Sensor,LOAD,5V boost,5.000000,0.000000,0.015000,0.000000,0.075000,0.000000,100.000000,
5,RC filter,LOSS,5V boost,5.000000,4.797050,0.006150,0.006150,0.000000,0.001248,0.959410,
6,LDO 2.5V,LINREG,RC filter,4.797050,2.500000,0.006150,0.006000,0.000000,0.014502,0.508443,
7,ADC,LOAD,LDO 2.5V,2.500000,0.000000,0.006000,0.000000,0.015000,0.000000,100.000000,
8,System total,,,3.000000,0.000000,0.021665,0.000000,0.064994,0.022994,0.646216,None


In [246]:
df =case1.solve()

In [247]:
len(df)

9

In [260]:
r = df[df['Element'] == 'System total']['Efficiency (%)'][8]
r

0.6462157028256803

In [259]:
r[8]

0.6462157028256803

In [262]:
t = case1.tree()

In [264]:
type(t)

rich.tree.Tree

In [60]:
l = [0, -1, 3, [3,4], 6]
l

[0, -1, 3, [3, 4], 6]

In [15]:
a = -np.ones (7)
a

array([-1., -1., -1., -1., -1., -1., -1.])

In [14]:
a[3] = [3,4]

ValueError: setting an array element with a sequence.

In [57]:
l.reverse()

In [61]:
l[::-1]

[6, [3, 4], 3, -1, 0]

In [59]:
l

[6, [3, 4], 3, -1, 0]